# Reading JSON input file and extracting data

In [35]:
import json   
import pandas as pd

In [36]:
f1 = open("Sample_Data.json" , 'r')

lines = f1.readlines()

tweets = {}
tweets['tweets'] = []

for line in lines:
    if line[0] == '{':
        tweets['tweets'].append(json.loads(line)) 

In [37]:
tweet = tweets['tweets']
tweet[0]['user']

{'id': 1242817830946508801,
 'id_str': '1242817830946508801',
 'name': 'juwelz v',
 'screen_name': 'juwelz_v',
 'location': 'Lower East Side, Manhattan',
 'url': 'https://linktr.ee/juwelzv',
 'description': 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 43,
 'friends_count': 118,
 'listed_count': 0,
 'favourites_count': 722,
 'statuses_count': 906,
 'created_at': 'Wed Mar 25 14:17:28 +0000 2020',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': '',
 'profile_background_image_url_https': '',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile

In [38]:
#df_twitter = pd.DataFrame(tweet)

In [39]:
#df_twitter.head(2)

In [40]:
#df_twitter_user = df_twitter[['user']]
#df_twitter_user.head(5)

In [41]:
tweet[0]['user']['id_str']


'1242817830946508801'

In [42]:
len(tweet)

2001

# Connection to postgresql database

In [74]:
import psycopg2 # psycopg2 is a PostgreSQL database adapter for the Python , and it provides a Python interface for connecting to and interacting with PostgreSQL databases.

In [75]:

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="TwitterDatabase",
    user="postgres",
    password="India@2194"
)


### Creating a cursor object to execute PostgreSQL commands

In [76]:
# Create a cursor object to execute PostgreSQL commands
cursor = conn.cursor()

# Inserting input data into the relational database

In [77]:
c=0
for i in range(0,len(tweet)):
    try:
        #print(i)
        #print(tweet[i]['user']['id_str'])
        #print(tweet[i]['user']['id_str'])
        #print(tweet[i]['user']['screen_name'])
        #print(tweet[i]['user']['description'])
        #print(tweet[i]['user']['verified'])
        #print(tweet[i]['user']['followers_count'])
        #print(tweet[i]['user']['friends_count'])
        #print(tweet[i]['user']['listed_count'])
        #print(tweet[i]['user']['favourites_count'])
        #print(tweet[i]['user']['statuses_count'])
        #print(tweet[i]['user']['created_at'])
        #print(tweet[0]['user']['protected'] )
        cursor.execute("INSERT INTO users (id_str,screen_name,description,verified,followers_count,friends_count ,listed_count ,favourites_count,statuses_count,created_at ,protected) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (tweet[i]['user']['id_str'],tweet[i]['user']['screen_name'],tweet[i]['user']['description'],tweet[i]['user']['verified'],tweet[i]['user']['followers_count'],tweet[i]['user']['friends_count'],tweet[i]['user']['listed_count'],tweet[i]['user']['favourites_count'],tweet[i]['user']['statuses_count'],tweet[i]['user']['created_at'],tweet[i]['user']['protected'] ))
        print('Record inserted --------------------------------------------------------------------------------->')
        c=c+1
    except Exception as e:
        print("Error encountered:",e)
        print("Count:",c)
        
        # Roll back the transaction
        conn.rollback()
        continue
    #finally:
        #conn.commit()
# Commit changes to the database
conn.commit()



Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record ins

Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record ins

Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Error encountered: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (id_str)=(1241748141809991688) already exists.

Count: 1086
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted ----------------------------------------

Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Record inserted --------------------------------------------------------------------------------->
Error encountered: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (id_str)=(1215315322653200384) already exists.

Count: 1658
Record inserted --------------------------------------------------------------------------------->
Record inserted ----------------------------------------

In [78]:
print(c)

1851


In [105]:
# Close the cursor and database connection
cursor.close()
conn.close()
